First we import the necessary libraries

In [1]:
import urllib.request
import bs4 as bs
import time
import datetime

import pandas as pd
import numpy as np
import yfinance as yf


Then we get the ticker list of the SP500 companies

In [2]:
t_df = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

num_tickers = 500

tickers = list(t_df["Symbol"][0:num_tickers])

#tickers = ["TSLA",  "EBAY", "META" ]   # for test
#print(tickers)

Now we extract the titles of the articles and how long ago they were published.

In [3]:
sleep_time  = 5

def extract_dates(parsed_doc):
    from math import floor
    today = datetime.datetime.now().date()
    dates = parsed_doc.find_all('pubdate')
    list_date = []
    for date  in dates:
        d = datetime.datetime.strptime( date.text, "%a, %d %b %Y %H:%M:%S GMT" )
        list_date += [floor((today - d.date()).seconds / 1)]                          #.days
    return list_date


def get_titles(ticker, sleep_time, num_titles):
    '''
    This function return the url response from google news and parsed html for a stock ticker.
    The time parameter is the sleep time before each request
    '''
    url = 'https://news.google.com/rss/search?hl=en-US&q='+ticker+'&gl=US&ceid=US:en'

    time.sleep(sleep_time) ## wait 15 seconds between each request. This is SUPER IMPORTANT otherwise your IP-address will be banned for sending too frequent requests.

    doc = urllib.request.urlopen(url).read()
    parsed_doc = bs.BeautifulSoup(doc,'lxml')
    titles = parsed_doc.find_all('title')[1:]
    list_date = extract_dates(parsed_doc)                 # we  assume/have verified that the date are extracted in the same order as the titles
    dct = {"Ticker": [ ticker ]*len(titles) , "Titles" : titles, "Date" : list_date}
    title_df = pd.DataFrame(dct)
    title_df = title_df.sort_values(by="Date")
    return title_df.iloc[0:num_titles]

list_df = []                        #dictionary with input data
num_articles = 10
for ticker in tickers:
    title = get_titles(ticker, sleep_time, num_articles)
    list_df.append(title)
    #print(title)

#input_df = pd.DataFrame(dct.values(),index = tickers,columns=["Titles"])

C:\Users\volpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [4]:
df = list_df[0]
df.shape

input_df = pd.concat(list_df)
input_df.shape

(5000, 3)

In [5]:
#input_df[ input_df["Ticker"] == "EBAY" ]
#test_df = input_df
#test_df["NEW"] = [0]*test_df.shape[0]
#test_df.loc[   test_df["Ticker"] == "EBAY","NEW"] = 1
#test_df

In [6]:
sleep_time = 5
threshold = 0.01
threshold_2 = 0.008
threshold_3 = 0.006

def get_var_prc(ticker, sleep_time):
    tickerData = yf.Ticker(ticker)
    time.sleep(sleep_time)  # Wait between requests
    df = tickerData.history()
    if len(df) < 2:
        print(f"Insufficient data for {ticker}")
        return None
    df['Change'] = df['Close'].diff()
    return df["Change"].iloc[-1] / df["Close"].iloc[-2]  # Use iloc for integer indexing

def get_target(x):
    if x < -threshold:
        return -1
    elif abs(x) < threshold:
        return 0
    else:
        return 1
def get_target_2(x):
    if x < -threshold:
        return -1
    elif abs(x) < threshold_2:
        return 0
    else:
        return 1
def get_target_3(x):
    if x < -threshold:
        return -1
    elif abs(x) < threshold_3:
        return 0
    else:
        return 1


input_df_threshold_2 = input_df.copy()
input_df_threshold_3 = input_df.copy()

# Add a new column 'Var_prc' and assign values to it
for ticker in tickers:
    var_prc = get_var_prc(ticker, sleep_time)
    if var_prc is not None:  # Check if data is available
        input_df.loc[input_df["Ticker"] == ticker, 'Var_prc'] = var_prc
        input_df.loc[input_df["Ticker"] == ticker, 'Target'] = input_df[input_df["Ticker"] == ticker]['Var_prc'].apply(get_target)
        input_df_threshold_2.loc[input_df_threshold_2["Ticker"] == ticker, 'Var_prc'] = var_prc
        input_df_threshold_2.loc[input_df_threshold_2["Ticker"] == ticker, 'Target'] = input_df_threshold_2[input_df_threshold_2["Ticker"] == ticker]['Var_prc'].apply(get_target_2)
        input_df_threshold_3.loc[input_df_threshold_3["Ticker"] == ticker, 'Var_prc'] = var_prc
        input_df_threshold_3.loc[input_df_threshold_3["Ticker"] == ticker, 'Target'] = input_df_threshold_3[input_df_threshold_3["Ticker"] == ticker]['Var_prc'].apply(get_target_3)

BRK.B: No data found, symbol may be delisted


Insufficient data for BRK.B


BF.B: No price data found, symbol may be delisted (period=1mo)


Insufficient data for BF.B


In [7]:
print(input_df.tail())

   Ticker                                             Titles  Date   Var_prc  \
66    YUM  [Director Paget Alves Sells Shares of Yum Bran...     0  0.009608   
65    YUM  [Why Yum Brands had its first negative quarter...     0  0.009608   
64    YUM  [Here's Why Yum China Holdings (YUMC) is a Str...     0  0.009608   
63    YUM  [Yum! Brands, Inc. Just Missed Earnings - But ...     0  0.009608   
62    YUM  [YUM! Brands' (YUM) Stock Down on Q1 Earnings ...     0  0.009608   

    Target  
66     0.0  
65     0.0  
64     0.0  
63     0.0  
62     0.0  


https://amaiya.github.io/ktrain/text/index.html#ktrain.text.Transformer

In [9]:
input_df.to_csv("data_2705_comma.csv")
input_df.to_csv("data_2705_semicolon.csv", sep = ";")
input_df_threshold_2.to_csv("data_2705_semicolon_thresh_2.csv", sep = ";")
input_df_threshold_3.to_csv("data_2705_semicolon_tresh_3.csv", sep = ";")